# 🌍 NLLB-200 Translation Service — GPU T4

Lance le service de traduction et se connecte automatiquement à votre application.

**Une seule étape :** `Exécution → Tout exécuter`

In [ ]:
# ── Cellule 1 : Configuration ─────────────────────────────────────────────
# URL de votre application (ne pas modifier si vous utilisez le serveur par défaut)
VM_URL = "http://144.126.203.233:3001"
# Token de sécurité (doit correspondre à COLAB_SECRET dans le backend)
COLAB_SECRET = "docuLens2024"

In [ ]:
# ── Cellule 2 : Installation ──────────────────────────────────────────────
!pip install -q transformers sentencepiece torch fastapi uvicorn nest_asyncio
!curl -sL https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
print('✅ Dépendances installées')

In [ ]:
# ── Cellule 3 : Service FastAPI ───────────────────────────────────────────
service_code = '''
import os, logging, asyncio
from contextlib import asynccontextmanager
from typing import List
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger(__name__)

MODEL_NAME  = os.getenv("NLLB_MODEL", "facebook/nllb-200-distilled-600M")
MAX_LENGTH  = int(os.getenv("NLLB_MAX_LENGTH", "512"))
BATCH_SIZE  = int(os.getenv("NLLB_BATCH_SIZE", "32"))
PORT        = int(os.getenv("TRANSLATION_PORT", "5001"))

SUPPORTED_LANGUAGES = {
    "fra_Latn": "Français",
    "eng_Latn": "Anglais",
    "arb_Arab": "Arabe (standard)",
    "hau_Latn": "Haoussa",
    "yor_Latn": "Yoruba",
    "swh_Latn": "Swahili",
    "zho_Hans": "Chinois simplifié",
    "spa_Latn": "Espagnol",
    "por_Latn": "Portugais",
    "deu_Latn": "Allemand",
}

state = {"model": None, "tokenizer": None, "device": "cpu", "ready": False}

def load_model():
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
    log.info(f"Chargement {MODEL_NAME} sur {device.upper()}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )
    model.to(device).eval()
    state.update({"tokenizer": tokenizer, "model": model, "device": device, "ready": True})
    log.info(f"Modele pret sur {device.upper()}")

@asynccontextmanager
async def lifespan(app):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(None, load_model)
    yield

app = FastAPI(title="NLLB Translation Service", lifespan=lifespan)
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class TranslateRequest(BaseModel):
    texts: List[str]
    src_lang: str = "fra_Latn"
    tgt_lang: str = "eng_Latn"

class TranslateResponse(BaseModel):
    translations: List[str]
    src_lang: str
    tgt_lang: str
    count: int

def translate_batch(texts, src_lang, tgt_lang):
    import torch
    tokenizer, model, device = state["tokenizer"], state["model"], state["device"]
    results = []
    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i+BATCH_SIZE]
        non_empty_idx  = [j for j, t in enumerate(batch) if t.strip()]
        non_empty_text = [batch[j] for j in non_empty_idx]
        batch_results  = [""] * len(batch)
        if non_empty_text:
            tokenizer.src_lang = src_lang
            inputs = tokenizer(
                non_empty_text, return_tensors="pt",
                padding=True, truncation=True, max_length=MAX_LENGTH
            ).to(device)
            forced_bos = tokenizer.convert_tokens_to_ids(tgt_lang)
            with torch.no_grad():
                output_ids = model.generate(
                    **inputs, forced_bos_token_id=forced_bos,
                    max_length=MAX_LENGTH, num_beams=4
                )
            translated = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for idx, t in zip(non_empty_idx, translated):
                batch_results[idx] = t
        results.extend(batch_results)
    return results

@app.get("/health")
def health():
    return {"status": "ready" if state["ready"] else "loading", "model": MODEL_NAME, "device": state["device"]}

@app.get("/languages")
def languages():
    return {"languages": SUPPORTED_LANGUAGES}

@app.post("/translate", response_model=TranslateResponse)
async def translate(req: TranslateRequest):
    if not state["ready"]:
        raise HTTPException(503, detail="Modele en chargement")
    if req.src_lang == req.tgt_lang:
        raise HTTPException(400, detail="src_lang et tgt_lang doivent etre differents")
    log.info(f"Traduction {req.src_lang} -> {req.tgt_lang} | {len(req.texts)} textes")
    loop = asyncio.get_event_loop()
    translations = await loop.run_in_executor(None, translate_batch, req.texts, req.src_lang, req.tgt_lang)
    return TranslateResponse(
        translations=translations, src_lang=req.src_lang,
        tgt_lang=req.tgt_lang, count=len([t for t in translations if t])
    )

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")
'''
with open('/content/translation_service.py', 'w') as f:
    f.write(service_code)
print('✅ Service écrit')

In [ ]:
# ── Cellule 4 : Lancement uvicorn ─────────────────────────────────────────
import subprocess, time, requests, nest_asyncio, threading
nest_asyncio.apply()

proc = subprocess.Popen(
    ['uvicorn', 'translation_service:app', '--host', '0.0.0.0', '--port', '5001'],
    cwd='/content', stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)
threading.Thread(target=lambda: [print(l.decode('utf-8','replace'), end='') for l in proc.stdout], daemon=True).start()

print('⏳ Chargement du modèle sur GPU...')
for i in range(90):
    time.sleep(3)
    try:
        r = requests.get('http://localhost:5001/health', timeout=2)
        if r.json().get('status') == 'ready':
            print(f'\n✅ Modèle prêt sur {r.json()["device"].upper()}')
            break
    except: pass
else:
    print('❌ Timeout')

In [ ]:
# ── Cellule 5 : Tunnel Cloudflare + connexion automatique à la VM ─────────
import subprocess, re, requests, time

tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:5001'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)

PUBLIC_URL = None
print('⏳ Ouverture du tunnel...')
for line in tunnel.stdout:
    text = line.decode('utf-8', errors='replace')
    match = re.search(r'https://[\w-]+\.trycloudflare\.com', text)
    if match:
        PUBLIC_URL = match.group(0)
        break

print(f'\n🌐 URL : {PUBLIC_URL}')

# Envoyer automatiquement l'URL à la VM
print('\n📡 Connexion automatique à l\'application...')
connected = False
for attempt in range(5):
    try:
        r = requests.post(
            f'{VM_URL}/api/config/translation-url',
            json={'url': PUBLIC_URL, 'secret': COLAB_SECRET},
            timeout=10
        )
        if r.status_code == 200:
            print('\n' + '=' * 60)
            print('🚀 GPU Colab connecté à votre application !')
            print('   Retournez sur l\'app → le badge passe au vert')
            print('=' * 60)
            connected = True
            break
    except Exception as e:
        print(f'   Tentative {attempt+1}/5... ({e})')
        time.sleep(3)

if not connected:
    print('\n⚠️  Connexion automatique échouée.')
    print(f'   Copiez manuellement cette URL dans l\'app : {PUBLIC_URL}')